In [9]:
import numpy as np

In [13]:
def return_UEL_property(): 
    UEL_property = [
        "*******************************************************",
        "*UEL PROPERTY, ELSET=SOLID                             ",
        "210000., 0.3, 0.05, 2.7, 0.0127                        ",
        "** Young's modulus E, Poisson ratio nu, length scale lc", 
        "** critical energy release rate Gc, diffusivity D      ",
        "*******************************************************",
    ]
    return UEL_property


def return_user_element():
    USER_ELEMENT = [
        "*************************************************",
       f"*User element, nodes=8, type=U1, properties=5, coordinates=2, var=44",
        "1,2                                              ",
        "1,3                                              ",
        "1,11                                             ",
        "*************************************************"
    ]

    return USER_ELEMENT

def return_depvar():
    DEPVAR = [
        "*Depvar       ",
        "  11,         ",       
        "1, S11, S11   ",
        "2, S22, S22   ",
        "3, S33, S33   ",
        "4, S12, S12   ",
        "5, E11, E11   ",
        "6, E22, E22   ",
        "7, E33, E33   ",
        "8, E12, E12   ",
        "9, PHI, PHI   ",	
        "10, SH, SH   ",
        "11, CL, CL   "
    ]
    return DEPVAR

input_file_name = "phase_field_plate.inp"

# Open the file
with open(input_file_name, 'r') as fid:
    flines = fid.readlines()

# Process the lines
flines = [line.strip() for line in flines]

# strip function is for removing leading and trailing whitespaces

# Search the lines containing *ELEMENT
FFLINES = [line.strip().upper() for line in flines]
start_elements = [i for i, line in enumerate(FFLINES) if '*ELEMENT' in line and '*ELEMENT OUTPUT' not in line]
start_element_index = start_elements[0]
element_indices = [] # list of element index
element_connvectivity = [] # list of of list of nodes that make up the element

print("The starting element index is: ", start_element_index)
print(start_element_index)

start_mesh_index = start_element_index + 1

while FFLINES[start_mesh_index][0] != "*" and FFLINES[start_mesh_index][0] != " ":
    #print(FFLINES[start_mesh_index])
    # remove all empty spaces and split by comma
    # each line look like this: 1,    35,     2,    36,  2503,  5502,  5503,  5504,  5505
    split_line = FFLINES[start_mesh_index].replace(" ", "").split(",")
    #print(split_line)
    element_indices.append(split_line[0])
    element_connvectivity.append(split_line[1:])
    start_mesh_index += 1

# print("The element indices are: ", element_indices)
# print("The element connectivity are: ", element_connvectivity)

# Now we would count the number of elements 
num_elements = len(element_indices)
print("The number of elements is: ", num_elements)  

# We would start to reconstruct an identical mesh, except that we replace node indices with new indices
# New indices should start from a power of 10 that is at least greater than the number of elements
# For example, if num_elements = 59, then the new indices should start from 101 to 159
# If num_elements = 128, then the new indices should start from 1001 to 1128
# If num_elements = 3456, then the new indices should start from 10001 to 13456

# We would find the order of the number of elements
order = int(np.floor(np.log10(num_elements)))
offset = 10 * 10**order + 1
print("The offset is: ", offset)

new_element_indices = [i + offset for i in range(num_elements)]

print("The new element indices are: ", new_element_indices)

visualization_mesh = [
    "*ELEMENT, TYPE=CPE8R, ELSET=VISUALIZATION",
]

for i in range(num_elements):
    reconstructed_line = ", ".join([str(value) for value in [new_element_indices[i]] + element_connvectivity[i]])
    visualization_mesh.append(reconstructed_line)

print(visualization_mesh)


# Now, we would reconstruct the input file as follows

# 1. Replace the original *ELEMENT section line with this line
#    *Element, type=U1, elset=SOLID

# 2. Add the USER ELEMENT section just above the *ELEMENT section in step 1

# 3. Add the UEL property just below the element connectivity section of the original mesh *ELEMENT above

# 4. Add the visualization mesh just below the USER ELEMENT section

# 5. Finally, in **SECTION, we change to this 
# ** Section: Section-1
# *Solid Section, elset=VISUALIZATION, material=(whatever material you define here)

# 6. We would also modify the *Depvar section to include the key descriptions

# do step 1
import copy
flines_new = copy.deepcopy(flines)
flines_new[start_element_index] = "*ELEMENT, TYPE=U1, ELSET=SOLID"

end_element_index = start_element_index + num_elements + 1

# do step 2 to 4
USER_ELEMENT = return_user_element()
UEL_property = return_UEL_property()
flines_new = flines_new[:start_element_index] + USER_ELEMENT + flines_new[start_element_index:end_element_index] + UEL_property + visualization_mesh + flines_new[end_element_index:]

# do step 5
solid_section_index = [i for i, line in enumerate(flines_new) if '*SOLID SECTION' in line.upper()][0]

# we should change this line
# *Solid Section, elset=<whatever name>, material=<whatever name>
# to *Solid Section, elset=VISUALIZATION, material=<whatever name>

# find the index where the word material is found
starting_index_string = flines_new[solid_section_index].find("material")
print("The starting index of the word material is: ", starting_index_string)
flines_new[solid_section_index] = "*Solid Section, elset=VISUALIZATION, " + flines_new[solid_section_index][starting_index_string:]

# do step 6
DEPVAR = return_depvar()
# find the index of the *Depvar section
depvar_index = [i for i, line in enumerate(flines_new) if '*DEPVAR' in line.upper()][0]
flines_new = flines_new[:depvar_index] + DEPVAR + flines_new[depvar_index+2:]
# write this to see how it looks like
with open("phase_field_plate_uel.inp", 'w') as fid:
    for line in flines_new:
        fid.write(line + "\n")

The starting element index is:  16412
16412
The number of elements is:  5404
The offset is:  10001
The new element indices are:  [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017, 10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10041, 10042, 10043, 10044, 10045, 10046, 10047, 10048, 10049, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10072, 10073, 10074, 10075, 10076, 10077, 10078, 10079, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098, 10099, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 10109, 10110, 10111, 10112, 10113, 10114, 10115, 10116, 10117, 10118, 10119, 10120, 10121, 10122, 10123, 10124, 10